# Um modelo matemático para otimizar a sua vida na universidade

## Prof. Mayron César O. Moreira 

**Universidade Federal de Lavras (UFLA)**  
**Departamento de Ciência da Computação**  
**Lavras, Minas Gerais, Brasil**  

*Adaptado da III Semana da Matemática da UFLA (III SEMAT/UFLA)*  
*Lavras, MG, Brasil*

### Tabela de Horários

Por meio de uma planilha, complete sua tabela de horários com as atividades fixas de sua semana. O arquivo se chama **"Horarios.xlsx"**, e está no diretório do *notebook* desse curso. Alguns exemplos: $(i)$ os horários que você pretende se dedicar para as suas disciplinas; $(ii)$ horários de atividades recreativas ou esportivas; $(iii)$ reuniões de grupos de estudo, participação em projetos (estágio, IC, empresa Jr.).  

Vamos carregar as suas informações através de uma biblioteca do Python denominada *Pandas*. Assim, conseguiremos preencher os parâmetros do modelo matemático. Para padronizar os dados de entrada, insira a o símbolo **D-** antes de todas as disciplinas. Exemplo: **D-GCC218**, ou **D-01**.  

In [405]:
import pandas as pd; 
planilha1 = pd.read_excel('Horarios.xlsx', sheet_name='horario'); 
dados1 = planilha1.to_records(index=False)
    
planilha2 = pd.read_excel('Horarios.xlsx', sheet_name='carga')
dados2 = planilha2.to_records(index=False)
    
planilha3 = pd.read_excel('Horarios.xlsx', sheet_name='dias-atividade')
dados3 = planilha3.to_records(index=False)

### Contexto modelado

Nosso planejamento leva em conta os sete dias da semana, representados pelo conjunto $T$. Vamos considerar um conjunto de atividades $A$, divididos em um conjunto de disciplinas $D$ e um conjunto de atividades extras $E$ ($A = D \cup E$ e $D \cap E = \varnothing$). Os dias correspondentes a cada atividade é determinado pelo conjunto $T_a \subseteq T, a \in A$. Além disso, a carga horária semanal de $a \in A$ é dada por $w_a \in \mathbb{R}_+$. **Lembre-se:** a carga horária de uma disciplina $d \in D$ corresponde às horas que serão dedicadas. Utilizamos também um conjunto de *slots* de tempo $S = \{8, ..., 22\}$. Desejamos determinar um plano de atividades semanais tal que:  

* **Restrição 1**: as atividades pré-fixadas devem ter seus horários respeitados;  
* **Restrição 2**: cada horário (*slot*) deve ser ocupado por uma atividade (garante que não haja sobreposição de horários);  
* **Restrição 3**: a carga horária das atividades deve ser cumprida;  
* **Restrição 4**: os sábados à partir das 18h devem ser dedicados ao **lazer**;    

O objetivo de nosso modelo é obedecer todas as restrições e especificidades de nosso contexto, **maximizando** a carga horária de atividades de lazer! =)

### Parâmatros (*inputs*)

Façamos um resumo dos parâmetros necessários ao nosso modelo.  

* $D$: conjunto de disciplinas;  
* $E$: conjunto de atividades extras; 
* $A = D \cup E$: conjunto de atividades;  
* $\tilde{A}$: conjunto de triplas $(\overline{a},\overline{t},\overline{s})$ que correspondem a designações de atividades, hora e dia já pré-determinados;  
* $T$: dias da semana;  
* $S$: *slots* de tempo;  
* $T_a$: dias em que a atividade $a \in A$ deve ser realizada;  
* $w_a$: carga horária da atividade $a \in A$.  

Em Python, a implementação desses dados é apresentada abaixo, utilizando as informações preenchidas pela planilha.  

In [406]:
D = [x[0] for x in dados2 if 'D-' in x[0]]
E = [x[0] for x in dados2 if 'D-' not in x[0]]
E = E + ['Lazer'] # Acrescentando a atividade Lazer
A = D + E
T = [x for x in dados1[0] if x != 'Hora']
S = [x[0] for x in dados1 if x[0] != 'Hora']

# Criando um dicionário de atividade:carga horária
w = { x[0]:x[1] for x in dados2 }

# Criando a lista de dias fixos para cada atividades
linhaDias = dados1[0]
ativDiaHora = { a:[] for a in A } # Conjunto \tilde{A}
ativDia = { a:[] for a in A }
diaAtivHora = { d:[] for d in linhaDias if d != 'Hora'}
alimentacao = [] # Conjunto de dias e horários que iremos nos alimentar (pausa)
for a in A:
    for i in range(1,len(dados1)):
        for j in range(len(dados1[i])):
            if(a == dados1[i][j]):
                ativDiaHora[a].append((linhaDias[j],dados1[i][0]))
                
                if(linhaDias[j] not in ativDia[a]):
                    ativDia[a].append(linhaDias[j])
                
                diaAtivHora[linhaDias[j]].append((a,dados1[i][0]))
                
            elif((dados1[i][j] == 'Almoço' or dados1[i][j] == 'Janta')
                 and ((linhaDias[j], dados1[i][0]) not in alimentacao)):
                alimentacao.append((linhaDias[j], dados1[i][0]))
                
# Calculando as possibilidades de dias para cada atividade
Ta = { a:[] for a in A }
for x in dados3:
    for y in x:
        if(y in T):
            Ta[x[0]].append(y)

### Modelo Matemático

Para modelar nosso problema matematicamente, utilizaremos a biblioteca PuLP do Python 3, declarada na sequência.

In [407]:
from pulp import *

#### Variáveis de decisão

Vamos definir o conjunto de variáveis binárias $x_{ast} \in \{0,1\}$ igual a 1 se e somente a atividade $a \in A$ for alocada no slot de tempo $s \in S$, no dia $t \in T$.

In [408]:
x = LpVariable.dicts("x", (A, S, T), cat='Binary')

#### Modelo matemático

Criando o modelo de **maximização**.

In [409]:
modelo = LpProblem("Otimização de horários", LpMaximize)

#### Função objetivo

Como cada *slot* de tempo é discretizado de hora em hora, nossa função objetivo consiste em maximizar a quantidade de horas que a variável $x$ relativa ao *lazer* aparece.

\begin{equation}
\max \sum_{s \in S}\sum_{t \in T}x_{\overline{a}st}
\end{equation}

em que $\overline{a}$ = Lazer.

In [410]:
modelo += sum([x['Lazer'][s][t] for s in S for t in T])

#### Restrições 1: fixação de atividades definidas *a priori*

Seja a tripla $(\overline{a}, \overline{s}, \overline{t}) \in \tilde{A}$ como uma tripla de atividade, horário e dia de uma atividade fixa pertencentes a um conjunto de designações feitas *a priori*. Assim, nossa restrição pode ser escrita da seguinte forma: 

\begin{equation}
x_{\overline{a}\,\overline{s}\,\overline{t}} = 1, \forall (\overline{a},\overline{s},\overline{t}) \in \tilde{A}
\end{equation}

In [411]:
for a in A:
    for (t, s) in ativDiaHora[a]:
        modelo += x[a][s][t] == 1
        
for a in alimentacao:
    modelo += x['Alimentacao'][a[1]][a[0]] == 1

#### Restrições 2: cada *slot* de tempo deve ser ocupado por alguma atividade

\begin{equation}
\sum_{a \in A} x_{ast} = 1, \forall s \in S, \forall t \in T
\end{equation}

In [412]:
for s in S:
    for t in T:
        modelo += sum([x[a][s][t] for a in A]) == 1

#### Restrições 3: a carga horária de todas as atividades deve ser cumprida (a exceção do 'Lazer', que deve ser maximizado)

\begin{equation}
\sum_{s \in S}\sum_{t \in T_a} x_{ast} = w_a, \forall a \in A \backslash \{\mbox{'Lazer'}\}
\end{equation}

In [413]:
for a in A:
    if(a != 'Lazer' and a != 'Alimentacao'):
        modelo += sum([x[a][s][t] for s in S for t in Ta[a]]) == w[a]

#### Restrições 4: os sábados à partir das 18h devem ser dedicados ao **lazer**

\begin{equation}
x_{\tilde{a}s\tilde{t}} = 1, \forall s \in S, s \ge 18, \tilde{a} = \mbox{'Lazer'}, \tilde{t} = \mbox{'Sab'}
\end{equation}

In [414]:
for s in S:
    if(s >= 18 and s <= 22):
        modelo += x['Lazer'][s]['Sab'] == 1

#### Resolvendo o modelo

In [415]:
modelo.solve()

1

#### Status da solução encontrada

In [416]:
print("Status da resolução do modelo:", LpStatus[modelo.status])

Status da resolução do modelo: Optimal


#### Total de horas de lazer

In [417]:
print("Horas de Lazer = ", value(modelo.objective))

Horas de Lazer =  33.0


#### Gerando sua planilha de horários

In [418]:
# Create a Pandas dataframe from some data.

Horarios = { (s,t):' ' for s in S for t in T }

for a in A:
    for s in S:
        for t in T:
            if(value(x[a][s][t]) >= 0.9):
                Horarios[s,t] = a

dicionarioHoras = { 'Horarios': S }
dicionarioHoras1 = { t:[Horarios[s,t] for s in S] for t in T }

z = {**dicionarioHoras, **dicionarioHoras1}
df = pd.DataFrame(z)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Resultado1.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Horários', index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

### Novas restrições

Vamos aprimorar a solução desse modelo matemático. Consideremos as seguintes restrições adicionais:

* **Restrição 7**: as horas de estudo e de IC devem ser contínuas (sequenciais).

#### Restrições 7: as horas de estudo e de IC devem ser contínuas (sequenciais)

\begin{equation}
x_{ast} \le x_{a,s-1,t} + x_{a,s+1,t}, \forall a \in D \cup \{\mbox{'IC'}\}, \forall s \in S\backslash\{\mbox{'8','22'}\}, \forall t \in T_a
\end{equation}

\begin{equation}
x_{a,22,t} = x_{a,21,t}, \forall a \in D, \forall t \in T_a
\end{equation}

\begin{equation}
x_{a,7,t} = x_{a,8,t}, \forall a \in D, \forall t \in T_a
\end{equation}

In [419]:
for a in A:
    if(a != 'Alimentacao' and a != 'Lazer' and a != 'Esporte'):
        for t in Ta[a]:
            for s in S:
                if(s != 8 and s != 22):
                    if((t, s) not in ativDiaHora[a] and (t, s-1) not in ativDiaHora[a] and (t, s+1) not in ativDiaHora[a]):
                        modelo += x[a][s][t] <= x[a][s-1][t] + x[a][s+1][t]

                    elif((t, s) not in ativDiaHora[a] and (t, s-1) not in ativDiaHora[a]):
                        modelo += x[a][s][t] == x[a][s-1][t]

                    elif((t, s) not in ativDiaHora[a] and (t, s+1) not in ativDiaHora[a]):
                        modelo += x[a][s][t] == x[a][s+1][t]

            modelo += x[a][22][t] == x[a][21][t]
            modelo += x[a][8][t] == x[a][9][t]

#### Resolvendo o modelo

In [420]:
modelo.solve()

1

#### Status da solução encontrada

In [421]:
print("Status da resolução do modelo:", LpStatus[modelo.status])

Status da resolução do modelo: Optimal


#### Total de horas de lazer

In [422]:
print("Horas de Lazer = ", value(modelo.objective))

Horas de Lazer =  33.0


#### Gerando sua planilha de horários

In [423]:
# Create a Pandas dataframe from some data.

Horarios = { (s,t):' ' for s in S for t in T }

for a in A:
    for s in S:
        for t in T:
            if(value(x[a][s][t]) >= 0.9):
                Horarios[s,t] = a

dicionarioHoras = { 'Horarios': S }
dicionarioHoras1 = { t:[Horarios[s,t] for s in S] for t in T }

z = {**dicionarioHoras, **dicionarioHoras1}
df = pd.DataFrame(z)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Resultado2.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Horários', index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()


### Novas restrições - Versão 2

Aprimorando ainda mais a solução desse modelo.

* **Restrição 8**: no máximo duas disciplinas serão estudadas por dia. Para tanto, vamos definir a variável binária $y_{at}$ igual a 1 se a disciplina $a \in D$ aparece em algum momento do dia $t$. Através de uma função suficientemente grande $\mathcal{M}$, temos:

\begin{equation}
\sum_{s \in S} x_{ast} \le |S|y_{at} \qquad \forall a \in D, \forall t \in T_a\backslash\{\mbox{'Sab','Dom'}\}
\end{equation}

\begin{equation}
\sum_{a \in D}y_{at} \le 2 \qquad \forall t \in T\backslash\{\mbox{'Sab','Dom'}\}
\end{equation}

In [424]:
y = LpVariable.dicts("y", (D, T), cat='Binary')
for a in D:
    for t in Ta[a]:
        if(t != 'Sab' and t != 'Dom'):
            modelo += sum([x[a][s][t] for s in S]) <= len(S)*y[a][t]
            
for t in T:
    if(t != 'Sab' and t != 'Dom'):
        modelo += sum([y[a][t] for a in D]) <= 2

#### Resolvendo o modelo

In [425]:
modelo.solve()

1

#### Status da solução encontrada

In [426]:
print("Status da resolução do modelo:", LpStatus[modelo.status])

Status da resolução do modelo: Optimal


#### Total de horas de lazer

In [427]:
print("Horas de Lazer = ", value(modelo.objective))

Horas de Lazer =  33.0


#### Gerando sua planilha de horários

In [428]:
# Create a Pandas dataframe from some data.

Horarios = { (s,t):' ' for s in S for t in T }

for a in A:
    for s in S:
        for t in T:
            if(value(x[a][s][t]) >= 0.9):
                Horarios[s,t] = a

dicionarioHoras = { 'Horarios': S }
dicionarioHoras1 = { t:[Horarios[s,t] for s in S] for t in T }

z = {**dicionarioHoras, **dicionarioHoras1}
df = pd.DataFrame(z)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Resultado3.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Horários', index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

#### Enjoy it! =)